In [1]:
import numpy as np
from sklearn.model_selection import LeaveOneOut
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import os

2024-12-29 08:08:28.387240: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-29 08:08:29.056973: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# TensorFlow 로그 설정
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# GPU 설정 및 라이브러리 로드 확인
def configure_gpu():
    try:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print("GPU 설정 완료")
        else:
            print("GPU를 사용할 수 없습니다. TensorFlow 설치 및 환경을 확인하세요.")
    except Exception as e:
        print(f"GPU 설정 중 오류 발생: {e}")

configure_gpu()

GPU 설정 완료


2024-12-29 08:08:29.775291: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-29 08:08:29.814383: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-29 08:08:29.818326: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
# 데이터 경로 설정
data_dir = "/home/bcml1/sigenv/eeg_band_split"  # 실제 데이터 경로로 변경


In [4]:
# Load data function - assumes EEG data is preprocessed and split by participant
def load_data():
    # Placeholder function to load EEG data
    # X should be a 4D array: [samples, channels, height, width]
    # y should be a 1D array: [samples]
    # Replace this with actual data loading logic
    X = np.load('eeg_data.npy')
    y = np.load('eeg_labels.npy')
    participants = np.load('participant_ids.npy')  # Array indicating participant ID for each sample
    return X, y, participants

In [5]:
# Define CNN model structure
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [6]:
# Main script
if __name__ == "__main__":
    X, y, participants = load_data()
    
    num_classes = len(np.unique(y))
    input_shape = X.shape[1:]  # Assuming data is already reshaped appropriately
    y_categorical = to_categorical(y, num_classes=num_classes)

    loo = LeaveOneOut()
    accuracy_scores = []

    for train_index, test_index in loo.split(np.unique(participants)):
        train_participants = np.unique(participants)[train_index]
        test_participant = np.unique(participants)[test_index][0]

        # Split data by participants
        train_mask = np.isin(participants, train_participants)
        test_mask = participants == test_participant

        X_train, X_test = X[train_mask], X[test_mask]
        y_train, y_test = y_categorical[train_mask], y_categorical[test_mask]

        # Create and train model
        model = create_cnn_model(input_shape, num_classes)
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

        # Evaluate model
        test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
        accuracy_scores.append(test_accuracy)

        print(f"Test participant: {test_participant}, Test accuracy: {test_accuracy:.4f}")

    print(f"Mean accuracy across participants: {np.mean(accuracy_scores):.4f}")

FileNotFoundError: [Errno 2] No such file or directory: 'eeg_data.npy'

In [16]:
# 학습 결과 저장
model.save("eeg_cnn_model_mj.h5")
print("Model saved!")

Model saved!


In [8]:
 # 결과 평가
model = tf.keras.models.load_model("eeg_cnn_model_mj.h5")
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

NameError: name 'X_test' is not defined

In [ ]:
# 테스트 데이터 예측
test_sample = X_test[0]  # 임의의 샘플
test_sample = np.expand_dims(test_sample, axis=0)  # 배치 차원 추가
prediction = model.predict(test_sample)


#테스트 데이터로 코드를 예측하는 부분 구현 필요

# 결과 출력
print(f"Prediction (0=negative, 1=positive): {prediction[0][0]}")